In [ ]:
from pyrhyme import PyRhyme

from pathlib import Path

# !pip install numpy
import numpy as np

# !pip install astropy
from astropy import units as U
from astropy import constants as C
from astropy.cosmology import WMAP7

# !pip install scipy
from scipy.io import FortranFile

# !pip install plotly
import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates.default = "simple_white"

In [ ]:
# run wget -r –level=0 -E –ignore-length -x -k -p -erobots=off -np -N https://astronomy.sussex.ac.uk/\~iti20/RT_comparison_project/RT_workshop_data/T7_results/

RHYME_OUTPUT_DIR = './output'
ILIEV_7_DIR = 'astronomy.sussex.ac.uk/~iti20/RT_comparison_project/RT_workshop_data/T7_results'

IMAGE_DIR = './images'
Path(IMAGE_DIR).mkdir(parents=True, exist_ok=True)

BINARIES = {
    'CAPREOLE+C2-RAY': {
        'orig': {
            1: {'path': f'{ILIEV_7_DIR}/Mellema1.bin', },
            3: {'path': f'{ILIEV_7_DIR}/Mellema2.bin', },
            10: {'path': f'{ILIEV_7_DIR}/Mellema3.bin', },
            25: {'path': f'{ILIEV_7_DIR}/Mellema4.bin', },
            75: {'path': f'{ILIEV_7_DIR}/Mellema5.bin', },
        },
        'add': {
            1: {'path': f'{ILIEV_7_DIR}/Mellema_add1.bin', },
            3: {'path': f'{ILIEV_7_DIR}/Mellema_add2.bin', },
            10: {'path': f'{ILIEV_7_DIR}/Mellema_add3.bin', },
            25: {'path': f'{ILIEV_7_DIR}/Mellema_add4.bin', },
            75: {'path': f'{ILIEV_7_DIR}/Mellema_add5.bin', },
        },
    },
    'CORAL': {
        'orig': {
            1: {'path': f'{ILIEV_7_DIR}/coral1.bin', },
            3: {'path': f'{ILIEV_7_DIR}/coral2.bin', },
            10: {'path': f'{ILIEV_7_DIR}/coral3.bin', },
            25: {'path': f'{ILIEV_7_DIR}/coral4.bin', },
            75: {'path': f'{ILIEV_7_DIR}/coral5.bin', },
        },
        'add': {
            1: {'path': f'{ILIEV_7_DIR}/coral_add1.bin', },
            3: {'path': f'{ILIEV_7_DIR}/coral_add2.bin', },
            10: {'path': f'{ILIEV_7_DIR}/coral_add3.bin', },
            25: {'path': f'{ILIEV_7_DIR}/coral_add4.bin', },
            75: {'path': f'{ILIEV_7_DIR}/coral_add5.bin', },
        },
    },
    'FLASH-HC': {
        'orig': {
            1: {'path': f'{ILIEV_7_DIR}/flash1.bin', },
            3: {'path': f'{ILIEV_7_DIR}/flash2.bin', },
            10: {'path': f'{ILIEV_7_DIR}/flash3.bin', },
            25: {'path': f'{ILIEV_7_DIR}/flash4.bin', },
            75: {'path': f'{ILIEV_7_DIR}/flash5.bin', },
        },
        'add': {
            1: {'path': f'{ILIEV_7_DIR}/flash_add1.bin', },
            3: {'path': f'{ILIEV_7_DIR}/flash_add2.bin', },
            10: {'path': f'{ILIEV_7_DIR}/flash_add3.bin', },
            25: {'path': f'{ILIEV_7_DIR}/flash_add4.bin', },
            75: {'path': f'{ILIEV_7_DIR}/flash_add5.bin', },
        },
    },
    'LICORICE': {
        'orig': {
            1: {'path': f'{ILIEV_7_DIR}/Licorice1.bin', },
            3: {'path': f'{ILIEV_7_DIR}/Licorice2.bin', },
            10: {'path': f'{ILIEV_7_DIR}/Licorice3.bin', },
            25: {'path': f'{ILIEV_7_DIR}/Licorice4.bin', },
            75: {'path': f'{ILIEV_7_DIR}/Licorice5.bin', },
        },
        'add': {
            1: {'path': f'{ILIEV_7_DIR}/Licorice_add1.bin', },
            3: {'path': f'{ILIEV_7_DIR}/Licorice_add2.bin', },
            10: {'path': f'{ILIEV_7_DIR}/Licorice_add3.bin', },
            25: {'path': f'{ILIEV_7_DIR}/Licorice_add4.bin', },
            75: {'path': f'{ILIEV_7_DIR}/Licorice_add5.bin', },
        },
    },
    'RSPH': {
        'orig': {
            1: {'path': f'{ILIEV_7_DIR}/Susa1.bin', },
            3: {'path': f'{ILIEV_7_DIR}/Susa2.bin', },
            10: {'path': f'{ILIEV_7_DIR}/Susa3.bin', },
            25: {'path': f'{ILIEV_7_DIR}/Susa4.bin', },
            75: {'path': f'{ILIEV_7_DIR}/Susa5.bin', },
        },
        'add': {
            1: {'path': f'{ILIEV_7_DIR}/Susa_add1.bin', },
            3: {'path': f'{ILIEV_7_DIR}/Susa_add2.bin', },
            10: {'path': f'{ILIEV_7_DIR}/Susa_add3.bin', },
            25: {'path': f'{ILIEV_7_DIR}/Susa_add4.bin', },
            75: {'path': f'{ILIEV_7_DIR}/Susa_add5.bin', },
        },
    },
    'ZEUS-MP': {
        'orig': {
            1: {'path': f'{ILIEV_7_DIR}/Zeus1.bin', },
            3: {'path': f'{ILIEV_7_DIR}/Zeus2.bin', },
            10: {'path': f'{ILIEV_7_DIR}/Zeus3.bin', },
            25: {'path': f'{ILIEV_7_DIR}/Zeus4.bin', },
            75: {'path': f'{ILIEV_7_DIR}/Zeus5.bin', },
        },
        'add': {
            1: {'path': f'{ILIEV_7_DIR}/Zeus_add1.bin', },
            3: {'path': f'{ILIEV_7_DIR}/Zeus_add2.bin', },
            10: {'path': f'{ILIEV_7_DIR}/Zeus_add3.bin', },
            25: {'path': f'{ILIEV_7_DIR}/Zeus_add4.bin', },
            75: {'path': f'{ILIEV_7_DIR}/Zeus_add5.bin', },
        },
    },
}

Rhyme_CaseA = {
    1: {'path': f'./CaseA/Iliev-7-CaseA-000001.chombo.h5', },
    3: {'path': f'./CaseA/Iliev-7-CaseA-000003.chombo.h5', },
    10: {'path': f'./CaseA/Iliev-7-CaseA-000010.chombo.h5', },
    25: {'path': f'./CaseA/Iliev-7-CaseA-000025.chombo.h5', },
#     75: {'path': f'./CaseA/Iliev-7-CaseA-000075.chombo.h5', },
}

Rhyme_CaseB = {
    1: {'path': f'./CaseB/Iliev-7-CaseB-000001.chombo.h5', },
    3: {'path': f'./CaseB/Iliev-7-CaseB-000003.chombo.h5', },
    10: {'path': f'./CaseB/Iliev-7-CaseB-000010.chombo.h5', },
    25: {'path': f'./CaseB/Iliev-7-CaseB-000025.chombo.h5', },
#     75: {'path': f'./CaseB/Iliev-7-CaseB-000075.chombo.h5', },
}

# Check if files exist
def check_if_exist(path):
    if not Path(snap['path']).is_file():
        print(f"Not found: {snap['path']}")
        
for sim_name, sim in BINARIES.items():      
    for snap in sim['orig'].values():
        check_if_exist(snap['path'])
    for snap in sim['add'].values():
        check_if_exist(snap['path'])

for snap in Rhyme_CaseA.values():
    check_if_exist(snap['path'])
for snap in Rhyme_CaseB.values():
    check_if_exist(snap['path'])

In [ ]:
def reading_binary_files():
    Gamma = 5. / 3.
    result = {}
    
    for simname, sim in BINARIES.items():
        for snap_time, snap in sim['orig'].items():
            if snap_time not in result:
                result[snap_time] = {}
                
            if simname not in result[snap_time]:
                result[snap_time][simname] = {}
            
            f = FortranFile(snap['path'], 'r')
            domain = f.read_ints(np.int32)
            
            result[snap_time][simname]['nHI'] = f.read_reals(np.float32).reshape(domain)
            result[snap_time][simname]['p'] = f.read_reals(np.float32).reshape(domain)
            result[snap_time][simname]['T'] = f.read_reals(np.float32).reshape(domain)
            
            f.close()
            
        for snap_time, snap in sim['add'].items():
            if snap_time not in result:
                result[snap_time] = {}
                
            if simname not in result[snap_time]:
                result[snap_time][simname] = {}
                
            f = FortranFile(snap['path'], 'r')
            domain = f.read_ints(np.int32)
            
            result[snap_time][simname]['n'] = f.read_reals(np.float32).reshape(domain)
            result[snap_time][simname]['M'] = f.read_reals(np.float32).reshape(domain)
            result[snap_time][simname]['nHII'] = f.read_reals(np.float32).reshape(domain)
            
            f.close()
            
    for snap_time, snap in Rhyme_CaseB.items():
        if snap_time not in result:
            result[snap_time] = {}

        if 'Rhyme_CaseB' not in result[snap_time]:
            result[snap_time]['Rhyme_CaseB'] = {}
            
        r = PyRhyme(snap['path'])
        domain = r.dataset.problem_domain
        
        v = r.load_variables(silent=True)
        
        rho = v['rho'][0].reshape(domain) * (1 * v['rho'][1]).to(U.cm**-3).value
        vx = v['rho_u'][0].reshape(domain) / rho * (1 * U.Mpc / U.Myr).to(U.cm / U.s).value
        vy = v['rho_v'][0].reshape(domain) / rho * (1 * U.Mpc / U.Myr).to(U.cm / U.s).value
        vz = v['rho_w'][0].reshape(domain) / rho * (1 * U.Mpc / U.Myr).to(U.cm / U.s).value
        T_orig = v['temp'][0].reshape(domain)
        fHI = v['ntr_frac_0'][0].reshape(domain)
        p, _, T = r.calc_temperature(v, X=1.0, Y=0.0, Gamma=5./3.)
        T = T[0].reshape(domain)
        p = p.reshape(domain)
        p *= C.m_p.to(U.g).value * (1 * r.dataset.active['h5']['attrs']['pressure_unit']).to(U.cm**-3 * U.cm**2 / U.s**2).value
        
        M = np.sqrt(vx**2 + vy**2 + vz**2) / np.sqrt(Gamma * p / (C.m_p.to(U.g).value * rho))
        
        result[snap_time]['Rhyme_CaseB']['n'] = rho
        result[snap_time]['Rhyme_CaseB']['nHI'] = fHI
        result[snap_time]['Rhyme_CaseB']['nHII'] = 1.0 - fHI
        result[snap_time]['Rhyme_CaseB']['T'] = T_orig
        result[snap_time]['Rhyme_CaseB']['p'] = p
        result[snap_time]['Rhyme_CaseB']['M'] = M
        
        r.dataset.close_current()
        r.dataset.clean_all()
        
    for snap_time, snap in Rhyme_CaseA.items():
        if snap_time not in result:
            result[snap_time] = {}

        if 'Rhyme_CaseA' not in result[snap_time]:
            result[snap_time]['Rhyme_CaseA'] = {}
            
        r = PyRhyme(snap['path'])
        domain = r.dataset.problem_domain
        
        v = r.load_variables(silent=True)
        
        rho = v['rho'][0].reshape(domain) * (1 * v['rho'][1]).to(U.cm**-3).value
        vx = v['rho_u'][0].reshape(domain) / rho * (1 * U.Mpc / U.Myr).to(U.cm / U.s).value
        vy = v['rho_v'][0].reshape(domain) / rho * (1 * U.Mpc / U.Myr).to(U.cm / U.s).value
        vz = v['rho_w'][0].reshape(domain) / rho * (1 * U.Mpc / U.Myr).to(U.cm / U.s).value
        T_orig = v['temp'][0].reshape(domain)
        fHI = v['ntr_frac_0'][0].reshape(domain)
        p, _, T = r.calc_temperature(v, X=1.0, Y=0.0, Gamma=5./3.)
        T = T[0].reshape(domain)
        p = p.reshape(domain)
        p *= C.m_p.to(U.g).value * (1 * r.dataset.active['h5']['attrs']['pressure_unit']).to(U.cm**-3 * U.cm**2 / U.s**2).value
        
        M = np.sqrt(vx**2 + vy**2 + vz**2) / np.sqrt(Gamma * p / (C.m_p.to(U.g).value * rho))
        
        result[snap_time]['Rhyme_CaseA']['n'] = rho
        result[snap_time]['Rhyme_CaseA']['nHI'] = fHI
        result[snap_time]['Rhyme_CaseA']['nHII'] = 1.0 - fHI
        result[snap_time]['Rhyme_CaseA']['T'] = T_orig
        result[snap_time]['Rhyme_CaseA']['p'] = p
        result[snap_time]['Rhyme_CaseA']['M'] = M
        
        r.dataset.close_current()
        r.dataset.clean_all()
    
    return result
            
    
if __name__ == '__main__' and '__file__' not in globals():
    DATA = reading_binary_files()

In [ ]:
def line_plot(attr='nHI', attr_title='X<sub>HI</sub>'):
    width = 700
    height = 500
    
    for t in DATA.keys():
        print(f'time: {t} Myr')
        fig = go.Figure()
        
        for sn, sim in DATA[t].items():
            if sn in ['Rhyme_CaseA', 'Rhyme_CaseB']:
                continue

            shape = sim[attr].shape
            
            if sn == 'ZEUS-MP':
                y = sim[attr][:, int(shape[1]/2), int(shape[2]/2)]
            else:
                y = sim[attr][int(shape[0]/2), int(shape[1]/2), :]
            
            fig.add_trace(go.Scatter(
                y=y, name=sn,
                mode='lines', line=dict(width=0.5),
            ))
            
        shape = DATA[t]['Rhyme_CaseA'][attr].shape
        fig.add_trace(go.Scatter(
            y=DATA[t]['Rhyme_CaseA'][attr][int(shape[0]/2), int(shape[1]/2), :], name='Rhyme_CaseA',
            mode='lines', line=dict(color='green', width=1.5),
        ))
            
        shape = DATA[t]['Rhyme_CaseB'][attr].shape
        fig.add_trace(go.Scatter(
            y=DATA[t]['Rhyme_CaseB'][attr][int(shape[0]/2), int(shape[1]/2), :], name='Rhyme_CaseB',
            mode='lines', line=dict(color='black', width=2.5),
        ))

        fig.update_layout(
            # title=f't = {t} Myr',
            width=width, height=height,
            margin=dict(l=10,r=10,b=10,t=10),
            xaxis=dict(mirror=True, type='linear', title='x [px]', exponentformat='power',),
            yaxis=dict(mirror=True, type='log', title=attr_title, exponentformat='power',),
            legend=dict(
                xanchor='left', yanchor='top', x=1.04, y=0.99
            ), showlegend=True,
        )
        
        fig.write_image(f'{IMAGE_DIR}/{attr}-{t}.png', width=width, height=height, scale=3)
        fig.write_image(f'{IMAGE_DIR}/{attr}-{t}.svg', width=width, height=height)
        fig.show()
        
    
if __name__ == '__main__' and '__file__' not in globals():
    line_plot(attr='n', attr_title='<i>n</i> [cm<sup>-3</sup>]')
    line_plot(attr='p', attr_title='<i>p</i> [cgs]')
    line_plot(attr='T', attr_title='<i>T</i> [K]')
    line_plot(attr='nHI', attr_title='f<sub>HI</sub>')
    line_plot(attr='nHII', attr_title='f<sub>HII</sub>')
    line_plot(attr='M', attr_title='<i>M</i> = |<i>v</i>| / <i>c</i><sub>s</sub>')

In [ ]:
def surface_plot(attr='nHI', attr_title='X<sub>HI</sub>', colorscale='Spectral'):
    width = 1000
    height = 380
    
    for t in DATA.keys():
        fig = make_subplots(
            2, 4,
            column_widths=[2, 1, 1, 1], row_heights=[1, 1],
            specs=[
                [{"type": "heatmap", "rowspan": 2}, {"type": "heatmap"}, {"type": "heatmap"}, {"type": "heatmap"}],
                [            None                 , {"type": "heatmap"}, {"type": "heatmap"}, {"type": "heatmap"}]
            ],
            vertical_spacing=0.05, horizontal_spacing=0.025,
            shared_xaxes=True, shared_yaxes=True,
            x_title='x [px]', y_title='y [px]',
        )
        
        zmin = np.nanmin([np.nanmin(x[attr]) for x in DATA[t].values()])
        zmax = np.nanmax([np.nanmax(x[attr]) for x in DATA[t].values()])
        
        for ip, (sn, sim) in enumerate(DATA[t].items()):
            if sn == 'Rhyme_CaseA':
                continue
            if sn == 'Rhyme_CaseB':
                irow, icol = 1, 1
            elif sn == 'CAPREOLE+C2-RAY':
                irow, icol = 1, 2
            elif sn == 'RSPH':
                irow, icol = 1, 3
            elif sn == 'ZEUS-MP':
                irow, icol = 1, 4
            elif sn == 'LICORICE':
                irow, icol = 2, 2
            elif sn == 'FLASH-HC':
                irow, icol = 2, 3
            elif sn == 'CORAL':
                irow, icol = 2, 4
            else:
                print('Unknown sim!')
                continue
                
            shape = sim[attr].shape
            
            if sn == 'ZEUS-MP':
                z = sim[attr][:, int(shape[2]/2), :].transpose()
            else:
                z = sim[attr][int(shape[1]/2), :, :]
            
            fig.add_trace(go.Heatmap(
                z=np.log10(z),
                zauto=False, zmin=np.log10(zmin), zmax=np.log10(zmax),
                colorbar=dict(
                    title=attr_title, titleside='bottom', titlefont=dict(size=18),
                    thicknessmode='pixels', thickness=10, tickfont=dict(size=18),
                ),
                colorscale=colorscale, reversescale=True, showscale=True if ip == 0 else False,
            ), irow, icol)
            
            if sn != 'Rhyme':
                fig.update_yaxes(showticklabels=False, row=irow, col=icol)
                
        annot_color = 'black' if attr in ['p', 'M'] else 'white'
        fig.add_annotation(
            xref='x1', yref='y1', xanchor='left', yanchor='bottom', x=5, y=3, text='Rhyme',
            showarrow=False, font=dict(color=annot_color, size=20),
        )
        fig.add_annotation(
            xref='x2', yref='y2', xanchor='left', yanchor='bottom', x=5, y=3, text='CAPREOLE+C2-RAY',
            showarrow=False, font=dict(color=annot_color, size=15),
        )
        fig.add_annotation(
            xref='x3', yref='y3', xanchor='left', yanchor='bottom', x=5, y=3, text='RSPH',
            showarrow=False, font=dict(color=annot_color, size=15),
        )
        fig.add_annotation(
            xref='x4', yref='y4', xanchor='left', yanchor='bottom', x=5, y=3, text='ZEUS-MP',
            showarrow=False, font=dict(color=annot_color, size=15),
        )
        fig.add_annotation(
            xref='x5', yref='y5', xanchor='left', yanchor='bottom', x=5, y=3, text='LICORICE',
            showarrow=False, font=dict(color=annot_color, size=15),
        )
        fig.add_annotation(
            xref='x6', yref='y6', xanchor='left', yanchor='bottom', x=5, y=3, text='FLASH-HC',
            showarrow=False, font=dict(color=annot_color, size=15),
        )
        fig.add_annotation(
            xref='x7', yref='y7', xanchor='left', yanchor='bottom', x=5, y=3, text='CORAL',
            showarrow=False, font=dict(color=annot_color, size=15),
        )
            
        fig.update_xaxes(
            mirror=True,
            tickmode='array', tickvals=[0, 64, 127], ticktext=['0', '3.3', '6.6'],
        )
        
        fig.update_yaxes(
            mirror=True,
            tickmode='array', tickvals=[0, 64, 127], ticktext=['0', '3.3', '6.6'],
        )
        
        fig.update_layout(
            # title=f't = {t} Myr',
            width=width, height=height,
            margin=dict(l=60,r=10,b=60,t=10),
            legend=dict(
                xanchor='left', yanchor='top', x=0.04, y=0.99
            ),
        )
        
        fig.write_image(f'{IMAGE_DIR}/heatmap-{attr}-{t}.png', width=width, height=height, scale=3)
        fig.write_image(f'{IMAGE_DIR}/heatmap-{attr}-{t}.svg', width=width, height=height)
        fig.show()
        
    
if __name__ == '__main__' and '__file__' not in globals():
    surface_plot(attr='n', attr_title='log <i>n</i>')
    surface_plot(attr='p', attr_title='log <i>p</i>')
    surface_plot(attr='T', attr_title='log <i>T</i>')
    surface_plot(attr='nHI', attr_title='log X<sub>HI</sub>')
    surface_plot(attr='nHII', attr_title='log X<sub>HII</sub>')
    surface_plot(attr='M', attr_title='log <i>M</i>')